In [1]:
# https://datamarket.com/data/set/22ox/monthly-milk-production-pounds-per-cow-jan-62-dec-75#!ds=22ox&display=line
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, TimeDistributed, Reshape
from tensorflow.keras.layers import Flatten, Dropout, BatchNormalization, Add, Subtract
from tensorflow.keras import layers
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from copy import copy
from tensorflow.keras import backend as K
from sklearn import preprocessing
import tensorflow.keras.optimizers as optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

## Separate splits

In [2]:
def augument_sample(sample, rate=0.1):
    return None
    
    
def format_data(df, lookback_window=12):
    x = []
    y = []
    for i in range(len(df)-lookback_window):
        x_entry = copy(df[i:i+lookback_window].values.T) # include current value
        x_entry[0][lookback_window-1] = 0.0 # erase variable to be predicted
        x.append(x_entry)
        y_entry = df['total_cases'][i+lookback_window-1]
        y.append(y_entry)
        
    x = np.array(x)
    y = np.array(y)
    return x, y

In [3]:
def split(x, y, val_size):
    train_size = len(x)-val_size
    x_train, y_train = x[0:train_size], y[0:train_size]
    x_val, y_val = x[train_size:train_size+val_size], y[train_size:train_size+val_size]
    
    assert y_train.shape[0]+y_val.shape[0] == len(y)
    
    return x_train, y_train, x_val, y_val

## Model with Autoregressive MLP

In [4]:
def get_model(input_shape):
    i = Input(shape=(input_shape))
    print(i.shape)
    m = i
    m =  Flatten()(m)
    m = Dense(500, activation='relu')(m)
    m = BatchNormalization()(m)
    m = Dense(200, activation='relu')(m)
    m = Dense(1, activation='relu')(m)

    model = Model(inputs=[i], outputs=[m])

    optimizer=optimizers.Adam(lr=5e-6)
    model.compile(optimizer, 'mae')
    return model

In [5]:
import os
import sys

sys.path.insert(0, './CLR')
sys.path.insert(0, './keras-tcn')

from tcn import TCN

In [6]:
from clr_callback import CyclicLR




def train_model(model, data, epochs=200, plot=True, verbose=False, save_checkpoints=False):
    start_time = time.time()
    print('Training...')
    x_train, y_train, x_val, y_val = data
    validation_data = (x_val, y_val)
    
    clr = CyclicLR(base_lr=1e-6, max_lr=1e-4, step_size=3000., mode='triangular2')
    
    callbacks = [clr]
    if save_checkpoints:
        callbacks.append(ModelCheckpoint("temp.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
                        )
    history = model.fit(x_train, y_train, epochs=epochs,
                        verbose=verbose, validation_data=validation_data, batch_size=50,
                        callbacks=callbacks)
    elapsed_time = time.time() - start_time
    
    print("Elapsed Time: {}".format(elapsed_time))
    
    if plot:
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
#         plt.ylim(0, 40)
        plt.legend(loc='upper right')
        plt.show()
    


In [7]:
def plot_train_val(model, data):
    x_train, y_train, x_val, y_val = data
    p = model.predict(x_train)
    plt.plot(y_train, label='actual')
    plt.plot(p, label='predicted')
    plt.title('Weekly Dengue Cases')
    plt.legend()
    plt.show()

    p = model.predict(x_val)
    plt.plot(y_val, label='actual')
    plt.plot(p, label='predicted')
    plt.title('Weekly Dengue Cases')
    plt.legend()
    plt.show()
    


## Forecasting

In [8]:
def forecast(model, forecast_window, last_train, test_data):
    lookback_window = last_train.shape[1]
    current = np.expand_dims(last_train, axis=0)
    next_y = model.predict(current)[0][0]
    current[0][0][-1] = next_y
    
    result = []
    for xi in test_data:
        xi[0][0:lookback_window] = current[0][0][-lookback_window:]
        current = np.expand_dims(xi, axis=0)
        next_y = model.predict(current)[0][0]
        current[0][0][-1] = next_y
        result.append(next_y)
    
    return result
    
def fake_test_data(x_val):
    x_test = copy(x_val)
    for i in x_test:
        i[0] = 0.0
    return x_test

def forecast_analysis(model, data):
    x_train, y_train, x_val, y_val = data
    x_test = fake_test_data(x_val)
    result = forecast(model, len(x_test), copy(x_train[-1]), x_test)
    error = mean_absolute_error(y_val, result)
    print("Forecast MAE: {}".format(error))

    plt.plot(y_val, label='actual')
    plt.plot(result, label='forecast')
    plt.title('Weekly Dengue Cases Forecasting')
    plt.legend()
    plt.show()
    return result

## Load Data
- Load data from csv
- Merge dataframes
- Choose city
- Remove string columns

In [9]:
def load_data(lookback_window, city):
    labels = pd.read_csv('../data/dengue_labels_train.csv', parse_dates=True)
    features = pd.read_csv('../data/dengue_features_train.csv', parse_dates=True)

    all_data = pd.merge(labels, features, on=['city', 'year', 'weekofyear'])
    city_data = all_data[all_data.city == city]
    df = city_data.drop(['city', 'week_start_date'], axis=1)
    df = df.reset_index()
    df = df.drop(['index'], axis=1).sort_values(['year', 'weekofyear'], ascending=[True, True])

    # Move "total_cases" to column 0 to avoid bugs
    cols = list(df)
    cols.insert(0, cols.pop(cols.index('total_cases')))
    df = df.loc[:, cols]

    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

## Submission Results

In [10]:
def forecast_on_test_data(model, df_train_val, city, lookback_window):
    labels = pd.read_csv('../data/submission_format.csv', parse_dates=True)
    features = pd.read_csv('../data/dengue_features_test.csv', parse_dates=True)

    df_test = pd.merge(labels, features, on=['city', 'year', 'weekofyear'])

    df_test = df_test[df_test.city == city]
    df_test = df_test.drop(['city', 'week_start_date'], axis=1)
    len_test = len(df_test)
    
    df_full = pd.concat([df_train_val, df_test])
    df_full = df_full.reset_index()
    df_full = df_full.drop(['index'], axis=1).sort_values(['year', 'weekofyear'], ascending=[True, True])
    df_full.fillna(method='ffill', inplace=True)
    df_full.fillna(method='bfill', inplace=True)

    # Ensure the same order of columns
    cols = list(df_train_val)
    cols.insert(0, cols.pop(cols.index('total_cases')))
    df_full = df_full.loc[:, cols]

    x, y = format_data(df_full, lookback_window)
    x_train, y_train, x_test, y_test = split(x, y, len_test)
    data = x_train, y_train, x_test, y_test
#     plot_train_val(model, data)
    results = forecast_analysis(model, data)
    
    df_results = labels[labels.city == city].reset_index()
    df_results['total_cases'] = np.round(np.array(results)).astype(int)
    return df_results.drop(['index'], axis=1)

In [11]:
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# min_max_scaler.fit(x)
# x_scaled = min_max_scaler.transform(x)
# df = pd.DataFrame(x_scaled, columns=df.columns)

## Model / Train / Forecast

In [12]:
K.clear_session()

## San Juan

In [ ]:
from tensorflow.keras.layers import Lambda, Permute
from clr_callback import CyclicLR

train_split_percent = 0.7
lookback_window_sj = 100  # weeks.
df_sj = load_data(lookback_window_sj, city='sj')
x, y = format_data(df_sj, lookback_window_sj)
val_size = int(math.floor((len(x)*(1-train_split_percent))))
x_train, y_train, x_val, y_val = split(x, y, val_size)
data = x_train, y_train, x_val, y_val


def train_model(model, data, epochs=200, plot=True, verbose=False, save_checkpoints=False):
    start_time = time.time()
    print('Training...')
    x_train, y_train, x_val, y_val = data
    validation_data = (x_val, y_val)
    
#     clr = CyclicLR(base_lr=1e-6, max_lr=1e-4, step_size=3000., mode='triangular2')
    
    callbacks = []
    callbacks.append(CyclicLR(base_lr=1e-5, max_lr=3e-4, step_size=4000., mode='triangular2'))
    if save_checkpoints:
        callbacks.append(ModelCheckpoint("temp.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
                        )
    history = model.fit(x_train, y_train, epochs=epochs,
                        verbose=verbose, validation_data=validation_data, batch_size=50,
                        callbacks=callbacks)
    elapsed_time = time.time() - start_time
    
    print("Elapsed Time: {}".format(elapsed_time))
    
    if plot:
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
#         plt.ylim(0, 40)
        plt.legend(loc='upper right'=)
        plt.show()

def get_model_sj(input_shape):
    i = Input(shape=(input_shape))
    print(i.shape)
    m = i
    m = Permute((2, 1))(m)
    print(m.shape)
    m = Dropout(0.2)(m)
    
#     m = TCN(nb_filters=32,
#         nb_stacks=1,
#         kernel_size=4,
#         dilations=[2 ** i for i in range(8)])(m)
    
    print(m.shape)
    m = layers.Conv1D(128, 13, dilation_rate=4)(m)
    print(m.shape)
    m = BatchNormalization()(m)
    m = Flatten()(m)
    m = layers.GaussianNoise(0.15)(m)
    m = Dense(500, activation='relu')(m)
    m = BatchNormalization()(m)
    shortcut = m
    m = Dense(1000, activation='relu')(m)
    m = BatchNormalization()(m)
    m = Dense(500, activation='relu')(m)
    m = BatchNormalization()(m)
    m = Subtract()([m, shortcut])
    m = Dense(1, activation='relu')(m)

    model = Model(inputs=[i], outputs=[m])

    optimizer=optimizers.Adam(lr=5e-5)
    model.compile(optimizer, 'mae')
    return model

input_shape = (x.shape[1], x.shape[2])
model_sj = get_model_sj(input_shape)
train_model(model_sj, data, epochs=1800, verbose=2)
plot_train_val(model_sj, data)
forecast_analysis(model_sj, data)
df_results_sj = forecast_on_test_data(model_sj, df_sj, city='sj', lookback_window=lookback_window_sj)

(?, 23, 100)
(?, 100, 23)
(?, 100, 23)
(?, 52, 128)
Training...
Train on 586 samples, validate on 250 samples
Epoch 1/1800
586/586 - 2s - loss: 36.3828 - val_loss: 23.1040
Epoch 2/1800
586/586 - 0s - loss: 36.2821 - val_loss: 22.9475
Epoch 3/1800
586/586 - 0s - loss: 36.0798 - val_loss: 21.9435
Epoch 4/1800
586/586 - 0s - loss: 35.8073 - val_loss: 21.3444
Epoch 5/1800
586/586 - 0s - loss: 35.6632 - val_loss: 20.4156
Epoch 6/1800
586/586 - 0s - loss: 35.2809 - val_loss: 19.5417
Epoch 7/1800
586/586 - 0s - loss: 35.0877 - val_loss: 19.0314
Epoch 8/1800
586/586 - 0s - loss: 34.9334 - val_loss: 18.5360
Epoch 9/1800
586/586 - 0s - loss: 34.5625 - val_loss: 17.7076
Epoch 10/1800
586/586 - 0s - loss: 34.1689 - val_loss: 16.8694
Epoch 11/1800
586/586 - 0s - loss: 34.2675 - val_loss: 16.5443
Epoch 12/1800
586/586 - 0s - loss: 33.7045 - val_loss: 16.3466
Epoch 13/1800
586/586 - 0s - loss: 33.7406 - val_loss: 16.1864
Epoch 14/1800
586/586 - 0s - loss: 33.3481 - val_loss: 16.0378
Epoch 15/1800
586

Epoch 129/1800
586/586 - 0s - loss: 17.5995 - val_loss: 21.8842
Epoch 130/1800
586/586 - 0s - loss: 18.0191 - val_loss: 20.9967
Epoch 131/1800
586/586 - 0s - loss: 17.5814 - val_loss: 20.8541
Epoch 132/1800
586/586 - 0s - loss: 17.0590 - val_loss: 21.9046
Epoch 133/1800
586/586 - 0s - loss: 18.2222 - val_loss: 19.2771
Epoch 134/1800
586/586 - 0s - loss: 17.9300 - val_loss: 18.2598
Epoch 135/1800
586/586 - 0s - loss: 17.6541 - val_loss: 17.8405
Epoch 136/1800
586/586 - 0s - loss: 17.0889 - val_loss: 19.4980
Epoch 137/1800
586/586 - 0s - loss: 17.0117 - val_loss: 18.6152
Epoch 138/1800
586/586 - 0s - loss: 16.5228 - val_loss: 15.0839
Epoch 139/1800
586/586 - 0s - loss: 17.4352 - val_loss: 14.1041
Epoch 140/1800
586/586 - 0s - loss: 16.6698 - val_loss: 14.4956
Epoch 141/1800
586/586 - 0s - loss: 16.4685 - val_loss: 13.8749
Epoch 142/1800
586/586 - 0s - loss: 16.9964 - val_loss: 11.6237
Epoch 143/1800
586/586 - 0s - loss: 16.5442 - val_loss: 12.3833
Epoch 144/1800
586/586 - 0s - loss: 17.0

586/586 - 0s - loss: 9.4744 - val_loss: 20.3945
Epoch 258/1800
586/586 - 0s - loss: 9.7923 - val_loss: 18.4012
Epoch 259/1800
586/586 - 0s - loss: 11.3467 - val_loss: 17.3182
Epoch 260/1800
586/586 - 0s - loss: 11.4158 - val_loss: 14.1748
Epoch 261/1800
586/586 - 0s - loss: 10.2030 - val_loss: 12.6484
Epoch 262/1800
586/586 - 0s - loss: 10.1766 - val_loss: 13.4155
Epoch 263/1800
586/586 - 0s - loss: 9.7723 - val_loss: 11.3895
Epoch 264/1800
586/586 - 0s - loss: 11.0925 - val_loss: 23.4604
Epoch 265/1800
586/586 - 0s - loss: 12.0205 - val_loss: 32.3711
Epoch 266/1800
586/586 - 0s - loss: 9.9334 - val_loss: 17.6870
Epoch 267/1800
586/586 - 0s - loss: 10.9993 - val_loss: 12.1820
Epoch 268/1800
586/586 - 0s - loss: 10.4923 - val_loss: 15.5672
Epoch 269/1800
586/586 - 0s - loss: 10.0361 - val_loss: 15.0099
Epoch 270/1800
586/586 - 0s - loss: 10.6179 - val_loss: 12.5203
Epoch 271/1800
586/586 - 0s - loss: 10.6205 - val_loss: 15.3639
Epoch 272/1800
586/586 - 0s - loss: 10.1677 - val_loss: 13.

Epoch 387/1800
586/586 - 0s - loss: 6.6029 - val_loss: 14.3317
Epoch 388/1800
586/586 - 0s - loss: 7.0729 - val_loss: 16.4405
Epoch 389/1800
586/586 - 0s - loss: 7.3495 - val_loss: 16.2513
Epoch 390/1800
586/586 - 0s - loss: 7.0304 - val_loss: 13.2614
Epoch 391/1800
586/586 - 0s - loss: 6.2832 - val_loss: 15.0979
Epoch 392/1800
586/586 - 0s - loss: 6.2958 - val_loss: 14.1985
Epoch 393/1800
586/586 - 0s - loss: 7.7854 - val_loss: 13.7778
Epoch 394/1800
586/586 - 0s - loss: 6.7624 - val_loss: 14.2282
Epoch 395/1800
586/586 - 0s - loss: 6.7393 - val_loss: 13.9278
Epoch 396/1800
586/586 - 0s - loss: 6.6031 - val_loss: 16.8582
Epoch 397/1800
586/586 - 0s - loss: 6.9726 - val_loss: 17.8565
Epoch 398/1800
586/586 - 0s - loss: 6.9885 - val_loss: 17.0702
Epoch 399/1800
586/586 - 0s - loss: 7.1299 - val_loss: 13.4980
Epoch 400/1800
586/586 - 0s - loss: 6.3732 - val_loss: 12.7680
Epoch 401/1800
586/586 - 0s - loss: 7.2371 - val_loss: 12.7957
Epoch 402/1800
586/586 - 0s - loss: 6.5874 - val_loss: 

In [ ]:
plot_train_val(model_sj, data)
forecast_analysis(model_sj, data)
df_results_sj = forecast_on_test_data(model_sj, df_sj, city='sj', lookback_window=lookback_window_sj)

## Iquitos

In [ ]:
from tensorflow.keras.layers import Lambda, Permute
from clr_callback import CyclicLR

train_split_percent = 0.80
lookback_window_iq = 200  # weeks.
df_iq = load_data(lookback_window_iq, city='iq')
x, y = format_data(df_iq, lookback_window_iq)
val_size = int(math.floor((len(x)*(1-train_split_percent))))
x_train, y_train, x_val, y_val = split(x, y, val_size)
data = x_train, y_train, x_val, y_val


def train_model(model, data, epochs=200, plot=True, verbose=False, save_checkpoints=False):
    start_time = time.time()
    print('Training...')
    x_train, y_train, x_val, y_val = data
    validation_data = (x_val, y_val)
    
#     clr = CyclicLR(base_lr=1e-6, max_lr=1e-4, step_size=3000., mode='triangular2')
    
    callbacks = []
    callbacks.append(CyclicLR(base_lr=1e-6, max_lr=3e-4, step_size=5000., mode='triangular2'))
    if save_checkpoints:
        callbacks.append(ModelCheckpoint("temp.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
                        )
    history = model.fit(x_train, y_train, epochs=epochs,
                        verbose=verbose, validation_data=validation_data, batch_size=50,
                        callbacks=callbacks)
    elapsed_time = time.time() - start_time
    
    print("Elapsed Time: {}".format(elapsed_time))
    
    if plot:
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
#         plt.ylim(0, 40)
        plt.legend(loc='upper right')
        plt.show()

def get_model_iq(input_shape):
    i = Input(shape=(input_shape))
    print(i.shape)
    m = i
    m = Permute((2, 1))(m)
    print(m.shape)
    m = Dropout(0.25)(m)
    print(m.shape)
    m = layers.Conv1D(128, 7)(m)
    print(m.shape)
    m = BatchNormalization()(m)
    m = Flatten()(m)
    m = layers.GaussianNoise(0.15)(m)
    m = Dense(500, activation='relu')(m)
    m = BatchNormalization()(m)
    shortcut = m
    m = Dense(2000, activation='relu')(m)
    m = BatchNormalization()(m)
    m = Dense(500, activation='relu')(m)
    m = BatchNormalization()(m)
    m = Subtract()([m, shortcut])
    m = Dense(1, activation='relu')(m)

    model = Model(inputs=[i], outputs=[m])

    optimizer=optimizers.Adam(lr=5e-5)
    model.compile(optimizer, 'mae')
    return model

input_shape = (x.shape[1], x.shape[2])
model_iq = get_model_iq(input_shape)

train_model(model_iq, data, epochs=2500, verbose=2)

# model_iq = load_model("temp.h5")

plot_train_val(model_iq, data)
forecast_analysis(model_iq, data)
df_results_iq = forecast_on_test_data(model_iq, df_iq, city='iq', lookback_window=lookback_window_iq)

In [ ]:
plot_train_val(model_iq, data)
forecast_analysis(model_iq, data)
df_results_iq = forecast_on_test_data(model_iq, df_iq, city='iq', lookback_window=lookback_window_iq)

# Prepare Submission CSV

In [ ]:
submission_df = pd.concat([df_results_sj, df_results_iq])
submission_df = submission_df.sort_values(['city','year', 'weekofyear'],
                                                        ascending=[False, True, True])

labels = pd.read_csv('../data/submission_format.csv', parse_dates=True)

print(labels.index.equals(labels.index))  

labels['total_cases'] = submission_df['total_cases'].values #HACK, there is a mismatch in the data
labels.to_csv("submission.csv", index=False)